# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [54]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [55]:
!cat talks.tsv

title	category	type	url_slug	venue	date	location	talk_url	description	coauthors	citation
Priming boundaries in production: Data from French	talks	Conference talk	bevivino-SP-2024	Speech Prosody 2024 (SP2024)	2024-07-05	Leiden, The Netherlands 			M. Huygevelde, G. Turco & B. Hemforth	<strong>Bevivino, D.</strong>, Huygevelde, M., Hemforth, B., & Turco, G. (2024, July 5). Priming boundaries in production: Data from French. <em>12th International Conference on Speech Prosody. Special Session on Advances in studies on prosodic planning</em>. Leiden, The Netherlands. [Talk & <a href="https://www.isca-archive.org/speechprosody_2024/bevivino24_speechprosody.html">Proceedings paper</a>]
Priming prosodic boundaries across constructions 	talks	Conference talk	bevivino-LE-2022	Linguistic Evidence 2022	2022-10-06	Paris, France			G. Turco & B. Hemforth	<strong>Bevivino, D.</strong>, Turco, G., & Hemforth, B. (2022, October 6). Priming prosodic boundaries across constructions. <em>Linguistic Evidenc

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [56]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,category,type,url_slug,venue,date,location,talk_url,description,coauthors,citation
0,Priming boundaries in production: Data from Fr...,talks,Conference talk,bevivino-SP-2024,Speech Prosody 2024 (SP2024),2024-07-05,"Leiden, The Netherlands",NaN,NaN,"M. Huygevelde, G. Turco & B. Hemforth","<strong>Bevivino, D.</strong>, Huygevelde, M.,..."
1,Priming prosodic boundaries across constructions,talks,Conference talk,bevivino-LE-2022,Linguistic Evidence 2022,2022-10-06,"Paris, France",NaN,NaN,G. Turco & B. Hemforth,"<strong>Bevivino, D.</strong>, Turco, G., & He..."
2,Attention and labelling in monolingual and bil...,talks,Conference talk (delivered by MK),bevivino-ICIS-2022,the International Congress of Infant Studies (...,2022-07-09,"Ottawa, Canada",NaN,NaN,M. Kalashnikova & L. Singh,"Kalashnikova, M., <strong>Bevivino, D.</strong..."
3,Caregivers adjust their speech in response to ...,talks,Conference talk,bevivino-ABC-2021,ABC'21: Language and cognition in typical and ...,2021-12-04,Hong Kong/Virtual,NaN,NaN,L. Singh & M. Kalashnikova,"<strong>Bevivino, D.</strong>, Singh, L., & Ka..."
4,Priming cooperating prosodic phrasing increase...,posters,Poster presentation,bevivino-amlap-2025,Architectures and Mechanisms for Language Proc...,2025-09-04,"Prague, Czechia",NaN,NaN,B. Hemforth & G. Turco,"<strong>Bevivino, D.</strong>, Hemforth, B., &..."
5,Does priming prosodic phrasing modulate readin...,posters,Poster presentation,bevivino-amlap-2023,Architectures and Mechanisms for Language Proc...,2023-09-02,"Donostia-San Sebastián, Basque Country, Spain",NaN,NaN,G. Turco & B. Hemforth,"<strong>Bevivino, D.</strong>, Turco, G., & He..."
6,Data from online production experiments and ch...,posters,Poster presentation,bevivino-ICPhS-2023,the International Congress of Phonetic Science...,2023-08-10,"Prague, Czechia",NaN,NaN,B. Hemforth & G. Turco,"<strong>Bevivino, D.</strong>, Hemforth, B., &..."
7,Priming prosodic boundaries in an online speec...,posters,Poster presentation,bevivino-amlap-2022,Architectures and Mechanisms for Language Proc...,2022-09-07,"York, UK",NaN,NaN,B. Hemforth & G. Turco,"<strong>Bevivino, D.</strong>, Hemforth, B., &..."
8,Eye-tracking Workshop,invited,Invited tutorial,bevivino-RJCP-2025,Rencontres Jeunes Chercheurs en Parole (RJCP),2025-11-05,"Paris, France",NaN,NaN,M. Foegel,"<strong>Bevivino, D. </strong> & Foegel, Marti..."
9,Does priming prosodic phrasing modulate senten...,invited,Internal presentation,bevivino-XL-2023,the Experimental Linguistics Workshop,2023-09-08,"Paris, France",NaN,NaN,NaN,"<strong>Bevivino, D. </strong>(2023, September..."


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [57]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;",
    "–": "&ndash;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [58]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
    
    # ADD CITATION and coauthors
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    md += "\ncoauthors: '" + html_escape(item.coauthors) + "'"
    
    # MINIMAL ADDITION: add category if it exists
    if len(str(item.category)) > 0 and str(item.category) != 'nan':
        md += "\ncategory: " + str(item['category'])
    
    # close it
    md += "\n---"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [59]:
!ls ../_talks

2012-example-output.md		   2023-08-10-bevivino-ICPhS-2023.md
2021-12-04-bevivino-ABC-2021.md    2023-09-02-bevivino-amlap-2023.md
2022-01-17-bevivino-SFL-2023.md    2023-09-08-bevivino-XL-2023.md
2022-06-16-bevivino-Labex-2022.md  2024-06-18-bevivino-MPI-2024.md
2022-07-09-bevivino-ICIS-2022.md   2024-07-05-bevivino-SP-2024.md
2022-09-07-bevivino-amlap-2022.md  2025-09-04-bevivino-amlap-2025.md
2022-10-06-bevivino-LE-2022.md	   2025-11-05-bevivino-RJCP-2025.md
2023-07-12-bevivino-LSA-2023.md


In [60]:
!cat ../_talks/2013-03-01-tutorial-1.md

cat: ../_talks/2013-03-01-tutorial-1.md: No such file or directory
